In [32]:
# =========================================
# VWAPクロス × 残差フィルター（BTC基準）バックテスト
# リファクタ版：戦略パラメータは冒頭で一元管理
# =========================================

# 0) Libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# 1) Strategy / BT Parameters  ----------------------------------------------
# データ
BTC_csv = 'Market_Bybit_BTCUSDT_1min_20250901-20250926'
ALT_csv = 'Market_Bybit_ASTERUSDT_1min_20250901-20250926'
time_col = "timestamp"

# 残差（係数）推定
COEFF_WINDOW = 10           # 係数と残差スケールのローリング窓（分）
USE_MAD_SCALE = True       # True: MAD×1.4826 で標準化 / False: 標準偏差

# 残差フィルター
RESID_K = 1               # |z| > RESID_K で通過（例: 1.0/1.5/2.0）
RESID_PERSIST = 1           # 連続本数（1=一発、2=2本連続）
USE_DIRECTION_FILTER = False # 残差の符号とVWAPクロス方向の一致要求

# VWAP
VWAP_SHORT = 7
VWAP_MEDIUM = 14

# BT
INITIAL_CAPITAL = 1000.0
TAKER_FEE = 0.00035         # 片道手数料 (例: 0.035%)
NOTIONAL_SIZE = 500.0       # 名目USD
MIN_QTY = 0.0               # 取引所の最小数量（無ければ0）

# 2) IO & Prep  ------------------------------------------------------------
def load_market(csv, rename_prefix=None):
    df = pd.read_csv(f'01.data/{csv}.csv')
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(time_col)
    if rename_prefix:
        rename = {c: f"{c}_{rename_prefix}" for c in ["O","H","L","C","V"] if c in df.columns}
        df = df.rename(columns=rename)
    return df

df_btc   = load_market(BTC_csv)                 # timestamp, O/H/L/C/V (Cのみ使う)
df_altOHLCV = load_market(ALT_csv, "ALT")       # timestamp, O_ALT/H_ALT/L_ALT/C_ALT/V_ALT

# 3) 残差（ALT vs BTC）の算出  --------------------------------------------
def compute_residuals(df_alt_close: pd.DataFrame, df_btc_close: pd.DataFrame) -> pd.DataFrame:
    # 終値のみで係数推定（inner join）
    x = df_alt_close[[time_col, "C_ALT"]].merge(
        df_btc_close[[time_col, "C"]].rename(columns={"C":"C_BTC"}), on=time_col, how="inner"
    )
    # 1分logリターン
    x["ret_ALT"] = np.log(x["C_ALT"]).diff()
    x["ret_BTC"] = np.log(x["C_BTC"]).diff()
    x = x.dropna(subset=["ret_ALT","ret_BTC"]).reset_index(drop=True)

    # ローリングOLS：beta = Cov/Var, alpha = mean_ALT - beta*mean_BTC
    W = COEFF_WINDOW
    mean_ALT = x["ret_ALT"].rolling(W, min_periods=W).mean()
    mean_BTC = x["ret_BTC"].rolling(W, min_periods=W).mean()
    var_BTC  = x["ret_BTC"].rolling(W, min_periods=W).var(ddof=0)
    cov_AB   = x["ret_ALT"].rolling(W, min_periods=W).cov(x["ret_BTC"])

    x["beta_BTC"] = cov_AB / var_BTC
    x["alpha"]    = mean_ALT - x["beta_BTC"] * mean_BTC

    # 予想 & 残差
    x["ret_ALT_pred"] = x["alpha"] + x["beta_BTC"] * x["ret_BTC"]
    x["resid"] = x["ret_ALT"] - x["ret_ALT_pred"]

    # 標準化
    if USE_MAD_SCALE:
        def _mad(s):
            m = s.median()
            return 1.4826 * (np.abs(s - m)).median()
        scale = x["resid"].rolling(W, min_periods=W).apply(_mad, raw=False)
    else:
        scale = x["resid"].rolling(W, min_periods=W).std()
    x["resid_z"] = x["resid"] / scale

    return x[[time_col, "ret_ALT","ret_BTC","alpha","beta_BTC","resid","resid_z"]]

resid_df = compute_residuals(
    df_altOHLCV[[time_col,"C_ALT"]],
    df_btc[[time_col,"C"]]
)

# 4) ALT側でVWAPクロス & フィルター合成  ---------------------------------
def add_vwap_and_signal(df_alt: pd.DataFrame, resid_df: pd.DataFrame) -> pd.DataFrame:
    d = df_alt.merge(resid_df[[time_col,"resid_z"]], on=time_col, how="left").copy()

    # VWAP
    d["Typical_price"] = (d["O_ALT"] + d["H_ALT"] + d["L_ALT"] + d["C_ALT"]) / 4
    d["TPxV"] = d["Typical_price"] * d["V_ALT"]

    roll_tpv_s = d["TPxV"].rolling(VWAP_SHORT,  min_periods=1).sum()
    roll_v_s   = d["V_ALT"].rolling(VWAP_SHORT,  min_periods=1).sum()
    roll_tpv_m = d["TPxV"].rolling(VWAP_MEDIUM, min_periods=1).sum()
    roll_v_m   = d["V_ALT"].rolling(VWAP_MEDIUM, min_periods=1).sum()

    d["VWAP_short"]  = roll_tpv_s / roll_v_s
    d["VWAP_medium"] = roll_tpv_m / roll_v_m

    # クロス Regime（1: long bias / 0: short bias）
    d["Regeme_vwap"] = (d["VWAP_short"] > d["VWAP_medium"]).astype(int)
    reg_prev = d["Regeme_vwap"].shift(1)
    reg_changed = (d["Regeme_vwap"] != reg_prev).fillna(False)

    # 点灯バー
    d["Trade_ignittion_vwap"] = np.where(reg_changed, d["Regeme_vwap"], np.nan)

    # 残差フィルター
    gate_abs = d["resid_z"].abs() > RESID_K
    gate_persist = gate_abs.rolling(RESID_PERSIST, min_periods=RESID_PERSIST).sum() == RESID_PERSIST

    if USE_DIRECTION_FILTER:
        gate_dir = ((d["resid_z"] > 0) & (d["Regeme_vwap"] == 1)) | \
                   ((d["resid_z"] < 0) & (d["Regeme_vwap"] == 0))
        gate = gate_persist & gate_dir
    else:
        gate = gate_persist

    # 最終Trade（発火バー ∧ フィルター通過）
    d["Trade"] = np.where(d["Trade_ignittion_vwap"].notna() & gate, d["Regeme_vwap"], np.nan)

    # 参考：フィルター無し版（比較用）
    d["Trade_vwap_only"] = np.where(d["Trade_ignittion_vwap"].notna(), d["Regeme_vwap"], np.nan)
    return d

df_all = add_vwap_and_signal(df_altOHLCV, resid_df)

# 5) バックテスト  ---------------------------------------------------------
def run_backtest(
    df_in: pd.DataFrame,
    initial_capital: float = INITIAL_CAPITAL,
    fee: float = TAKER_FEE,
    notional_size: float = NOTIONAL_SIZE,
    min_qty: float = MIN_QTY
) -> pd.DataFrame:
    """名目サイズ固定・線形PnL・片道手数料・ドテン対応"""
    d = df_in.rename(columns={"C_ALT":"C","V_ALT":"V"}).copy()
    d = d[[time_col, "C", "V", "Trade"]].copy().sort_values(time_col).reset_index(drop=True)

    n = len(d)
    side = np.zeros(n, dtype=int)
    pos_qty = np.zeros(n)
    entry_price = np.full(n, np.nan)
    trade_qty = np.zeros(n)
    trade_notional = np.zeros(n)
    fee_paid = np.zeros(n)
    fee_cum = np.zeros(n)
    realized_pnl = np.zeros(n)
    realized_pnl_cum = np.zeros(n)
    unrealized_pnl = np.zeros(n)
    equity = np.zeros(n)

    cur_qty = 0.0
    cur_entry = np.nan
    cur_fee_cum = 0.0
    cur_realized = 0.0

    for i in range(n):
        price = float(d.at[i, "C"])
        sig = d.at[i, "Trade"]  # 1/0/NaN

        # 取引判定
        do_trade = False
        desired_side = 0
        if pd.notna(sig):
            desired_side = 1 if sig == 1 else -1
            if cur_qty == 0.0:
                do_trade = True
            else:
                cur_side = 1 if cur_qty > 0 else -1
                if desired_side == -cur_side:
                    do_trade = True

        # 約定
        if do_trade:
            # 既存クローズ
            if cur_qty != 0.0:
                close_qty = -cur_qty
                pnl_close = (price - cur_entry) * cur_qty
                cur_realized += pnl_close
                close_notional = abs(close_qty) * price
                fee_close = fee * close_notional
                cur_fee_cum += fee_close

                trade_qty[i] += close_qty
                trade_notional[i] += close_notional
                fee_paid[i] += fee_close
                realized_pnl[i] += pnl_close

                cur_qty = 0.0
                cur_entry = np.nan

            # 新規
            target_qty = notional_size / price
            if min_qty and min_qty > 0:
                target_qty = np.floor(target_qty / min_qty) * min_qty
            if target_qty > 0:
                open_qty = desired_side * target_qty
                open_notional = abs(open_qty) * price
                fee_open = fee * open_notional
                cur_fee_cum += fee_open

                trade_qty[i] += open_qty
                trade_notional[i] += open_notional
                fee_paid[i] += fee_open

                cur_qty = open_qty
                cur_entry = price

        # 評価
        u_pnl = (price - cur_entry) * cur_qty if cur_qty != 0 else 0.0
        unrealized_pnl[i] = u_pnl

        if i > 0:
            realized_pnl_cum[i] = realized_pnl_cum[i-1] + realized_pnl[i]
            fee_cum[i] = fee_cum[i-1] + fee_paid[i]
        else:
            realized_pnl_cum[i] = realized_pnl[i]
            fee_cum[i] = fee_paid[i]

        equity[i] = initial_capital + realized_pnl_cum[i] + unrealized_pnl[i] - fee_cum[i]

        side[i] = 0 if cur_qty == 0 else (1 if cur_qty > 0 else -1)
        pos_qty[i] = cur_qty
        entry_price[i] = cur_entry

    out = d.copy()
    out["side"] = side
    out["position_qty"] = pos_qty
    out["entry_price"] = entry_price
    out["trade_qty"] = trade_qty
    out["trade_notional"] = trade_notional
    out["fee_paid"] = fee_paid
    out["fee_paid_cum"] = fee_cum
    out["realized_pnl"] = realized_pnl
    out["realized_pnl_cum"] = realized_pnl_cum
    out["unrealized_pnl"] = unrealized_pnl
    out["equity"] = equity
    return out

# 6) 実行：フィルター無し vs 有り  ---------------------------------------
df_bt_A = df_all[[time_col,"C_ALT","V_ALT","Trade_vwap_only"]].rename(columns={"Trade_vwap_only":"Trade"})
df_bt_B = df_all[[time_col,"C_ALT","V_ALT","Trade"]]

bt_A = run_backtest(df_bt_A, INITIAL_CAPITAL, TAKER_FEE, NOTIONAL_SIZE, MIN_QTY)
bt_B = run_backtest(df_bt_B, INITIAL_CAPITAL, TAKER_FEE, NOTIONAL_SIZE, MIN_QTY)

# 7) Summary & Plot  -------------------------------------------------------
def max_dd_while_in_position(bt: pd.DataFrame, time_col: str) -> dict:
    """
    ポジション保有中だけを対象に、エクイティ曲線の最大DDを計測。
    戻り値:
      {
        'max_dd_frac': -0.1234,          # 比率 (負値)
        'max_dd_pct': -12.34,            # ％表示用
        'start_ts': Timestamp or None,   # そのDD区間の開始（エントリー時）
        'trough_ts': Timestamp or None,  # DDが最大になった時刻
      }
    """
    in_pos = bt['position_qty'].ne(0)
    peak = None
    run_start_ts = None
    max_dd = 0.0
    worst_start_ts = None
    worst_trough_ts = None

    for i, row in bt.iterrows():
        if in_pos.iloc[i]:
            eq = float(row['equity'])
            ts = row[time_col]

            if peak is None:
                peak = eq
                run_start_ts = ts

            if eq > peak:
                peak = eq

            dd = (eq / peak) - 1.0  # ≤ 0
            if dd < max_dd:
                max_dd = dd
                worst_start_ts = run_start_ts
                worst_trough_ts = ts
        else:
            # 区間リセット（フラットになったら別区間として扱う）
            peak = None
            run_start_ts = None

    return {
        'max_dd_frac': max_dd,
        'max_dd_pct' : max_dd * 100.0,
        'start_ts'   : worst_start_ts,
        'trough_ts'  : worst_trough_ts,
    }

def summary(bt: pd.DataFrame, tag: str):
    total_fee = bt['fee_paid'].sum()
    trade_count_bars = (bt['trade_qty'] != 0).sum()
    total_notional = bt['trade_notional'].sum()
    mdd_info = max_dd_while_in_position(bt, time_col)

    print(f"=== {tag} ===")
    print(f"Total fee paid       : {total_fee:.2f} USD")
    print(f"Number of trade bars : {trade_count_bars}")
    print(f"Total trade notional : {total_notional:.2f} USD")
    print(f"Final equity         : {bt['equity'].iloc[-1]:.2f} USD")
    print(f"Max DD (in-position) : {mdd_info['max_dd_pct']:.2f}% "
          f"(from {mdd_info['start_ts']} to {mdd_info['trough_ts']})")


summary(bt_A, "VWAP only")
summary(bt_B, "VWAP + Residual gate")

fig = go.Figure()
fig.add_trace(go.Scatter(x=bt_A[time_col], y=bt_A['equity'], mode='lines', name='VWAP only'))
fig.add_trace(go.Scatter(x=bt_B[time_col], y=bt_B['equity'], mode='lines', name='VWAP + Residual gate'))
fig.update_layout(title="Equity Curve Comparison", xaxis_title="Time", yaxis_title="Equity (USD)",
                  hovermode="x unified", template="plotly_white", height=520, width=1180)
fig.show()


=== VWAP only ===
Total fee paid       : 252.04 USD
Number of trade bars : 720
Total trade notional : 720107.31 USD
Final equity         : 107.73 USD
Max DD (in-position) : -98.07% (from 2025-09-19 09:25:00 to 2025-09-25 08:17:00)
=== VWAP + Residual gate ===
Total fee paid       : 55.37 USD
Number of trade bars : 158
Total trade notional : 158189.28 USD
Final equity         : 1470.69 USD
Max DD (in-position) : -14.78% (from 2025-09-19 10:22:00 to 2025-09-20 09:12:00)


In [33]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def dd_series_in_position(bt: pd.DataFrame, time_col: str):
    """
    保有中のみのDDを算出。
    戻り値:
      dd_df: DataFrame[time, equity, in_pos, run_id, dd_frac]  ※dd_frac≤0（NaNは非保有）
      max_dd_per_run: 各保有区間（ポジション保有連続区間）の最大DD（負値）の配列
    """
    in_pos = bt['position_qty'].ne(0).to_numpy()
    eq = bt['equity'].to_numpy()
    ts = bt[time_col].to_numpy()

    run_id = np.full(len(bt), -1, dtype=int)
    dd = np.full(len(bt), np.nan, dtype=float)

    cur_run = -1
    peak = None
    max_dd_this_run = 0.0
    max_dd_per_run = []

    for i in range(len(bt)):
        if in_pos[i]:
            if cur_run == -1:
                # 区間開始
                cur_run = (0 if (len(max_dd_per_run)==0 and run_id.max()==-1) else (run_id.max()+1))
                peak = eq[i]
                max_dd_this_run = 0.0
            else:
                peak = max(peak, eq[i])

            run_id[i] = cur_run
            dd_i = (eq[i] / peak) - 1.0
            dd[i] = dd_i
            if dd_i < max_dd_this_run:
                max_dd_this_run = dd_i

            # 最終バーで区切り処理
            if i == len(bt)-1:
                max_dd_per_run.append(max_dd_this_run)

        else:
            if cur_run != -1:
                # 区間終了
                max_dd_per_run.append(max_dd_this_run)
                cur_run = -1
                peak = None

    dd_df = pd.DataFrame({
        time_col: ts,
        "equity": eq,
        "in_pos": in_pos,
        "run_id": run_id,
        "dd_frac": dd,   # ≤0; 非保有はNaN
    })
    # 非保有のNaNは落としておく（ヒストグラム用）
    return dd_df, np.array(max_dd_per_run, dtype=float)

# ==== 使い方（例：残差フィルター版 bt_B）====
dd_df, max_dd_runs = dd_series_in_position(bt_B, time_col)

# 基本統計（％表記）
def pct(x): return x * 100.0
bar_dd = pct(dd_df['dd_frac'].dropna().to_numpy())
run_dd = pct(max_dd_runs[np.isfinite(max_dd_runs)])

print("---- In-position DD (per-bar) ----")
print(f"N={len(bar_dd)}, mean={bar_dd.mean():.2f}%, median={np.median(bar_dd):.2f}%, "
      f"p95={np.percentile(bar_dd,95):.2f}% (注意: DDは負値が通常)")

print("---- Max DD per run (per-position) ----")
if len(run_dd):
    print(f"N_runs={len(run_dd)}, worst={run_dd.min():.2f}%, "
          f"median={np.median(run_dd):.2f}%, p95={np.percentile(run_dd,95):.2f}%")
else:
    print("No positions/runs found.")

# ヒストグラム可視化（％）
fig = make_subplots(rows=1, cols=2, subplot_titles=("In-position DD (per-bar, %)", "Max DD per run (%)"))

fig.add_trace(
    go.Histogram(x=bar_dd, nbinsx=60, name="bar DD"),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=run_dd, nbinsx=30, name="run max DD"),
    row=1, col=2
)

# 見やすさ調整（DDは負側に山が来るはず）
fig.update_layout(
    title="Drawdown Distributions (while in position)",
    bargap=0.03, template="plotly_white", height=450, width=1100,
    showlegend=False
)
# x軸を負側中心に（任意で調整）
if len(bar_dd):
    lo1 = min(-0.1, np.nanpercentile(bar_dd, 1))  # 1%点 or -10%
    hi1 = max(0.0, np.nanpercentile(bar_dd, 99))
    fig.update_xaxes(range=[lo1, hi1], row=1, col=1)
if len(run_dd):
    lo2 = min(-10, np.nanpercentile(run_dd, 5))
    hi2 = max(0.0, np.nanpercentile(run_dd, 99))
    fig.update_xaxes(range=[lo2, hi2], row=1, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_xaxes(title_text="DD (%)", row=1, col=1)
fig.update_xaxes(title_text="DD (%)", row=1, col=2)

fig.show()


---- In-position DD (per-bar) ----
N=9219, mean=-4.59%, median=-4.15%, p95=-0.55% (注意: DDは負値が通常)
---- Max DD per run (per-position) ----
N_runs=1, worst=-14.78%, median=-14.78%, p95=-14.78%


In [34]:
bt_B

,timestamp,C,V,Trade,side,position_qty,entry_price,trade_qty,trade_notional,fee_paid,fee_paid_cum,realized_pnl,realized_pnl_cum,unrealized_pnl,equity
0,2025-09-19 09:25:00,0.7000,238145,NaN,0,0.000000,NaN,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,1000.000000
1,2025-09-19 09:26:00,0.7000,0,NaN,0,0.000000,NaN,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,1000.000000
2,2025-09-19 09:27:00,0.7000,0,NaN,0,0.000000,NaN,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,1000.000000
3,2025-09-19 09:28:00,0.7000,0,NaN,0,0.000000,NaN,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,1000.000000
4,2025-09-19 09:29:00,0.7000,0,NaN,0,0.000000,NaN,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9271,2025-09-25 19:56:00,1.9782,269564,NaN,-1,-251.268908,1.9899,0.0,0.0,0.0,55.366248,0.0,521.78724,2.939846,1469.360839
9272,2025-09-25 19:57:00,1.9710,242916,NaN,-1,-251.268908,1.9899,0.0,0.0,0.0,55.366248,0.0,521.78724,4.748982,1471.169975
9273,2025-09-25 19:58:00,1.9695,217691,NaN,-1,-251.268908,1.9899,0.0,0.0,0.0,55.366248,0.0,521.78724,5.125886,1471.546878
9274,2025-09-25 19:59:00,1.9704,168932,NaN,-1,-251.268908,1.9899,0.0,0.0,0.0,55.366248,0.0,521.78724,4.899744,1471.320736
